In [1]:
import pandas as pd
import numpy as np
import os
import datetime

In [2]:
os.chdir('C:\\Users\\Documentos\\Documents\\Estudos\\C1')
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 100)
nh = 3

In [3]:
Campanhas = pd.read_csv('Campanhas.csv', sep = '|')
print(Campanhas.head(nh))
print(Campanhas.info())

   Campaign_id                            Campaign_name
0            0                  Cupom_reforco_Industria
1            1             Cupom_reforco_Distribuidores
2            2  Incentivo1Compra_reforco_Distribuidores
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Campaign_id    80 non-null     int64 
 1   Campaign_name  80 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.4+ KB
None


In [4]:
Sent = pd.read_csv('Sent.csv', sep = '|')
Sent.columns = ['Campaign_id', 'DataEnvio', 'User_id']
print(Sent.head(nh))
print(Sent.info())

   Campaign_id            DataEnvio  User_id
0            0  2021-07-02 06:16:01    36588
1            0  2021-07-02 06:16:01    30846
2            0  2021-07-01 06:16:01    22542
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38400 entries, 0 to 38399
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Campaign_id  38400 non-null  int64 
 1   DataEnvio    38400 non-null  object
 2   User_id      38400 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 900.1+ KB
None


In [5]:
Orders = pd.read_csv('Orders.csv', sep = '|')
print(Orders.head(nh))
print(Orders.info())

    ORDER_ID  ORDER_STORE  ORDER_DATE  ORDER_TOTAL  User_id
0  460112019           61  2021-07-23         19.8    24637
1  460112020           61  2021-07-23       1309.0    26186
2  460112021           61  2021-07-23        467.4    28319
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102168 entries, 0 to 102167
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ORDER_ID     102168 non-null  int64  
 1   ORDER_STORE  102168 non-null  int64  
 2   ORDER_DATE   102168 non-null  object 
 3   ORDER_TOTAL  102168 non-null  float64
 4   User_id      102168 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 3.9+ MB
None


In [6]:
#Padronização das datas para o match. Teoricamente existe uma chance do acréscimo do Timedelta(1) skipar para o próximo dia, mas a chance é tão ínfima, que vou desconsiderar.
Orders['ORDER_DATE'] = pd.to_datetime(Orders['ORDER_DATE']) + pd.Timedelta(1)
Sent["DataEnvio"] = pd.to_datetime(Sent["DataEnvio"]) + pd.Timedelta(1)

#Medidas necessárias
aggOrders = Orders.groupby(['User_id', 'ORDER_DATE'])['ORDER_TOTAL'].agg(['size', 'sum']).reset_index()

#Matching
Sent.sort_values('DataEnvio', inplace = True)
aggOrders.sort_values('ORDER_DATE', inplace = True)
tol = pd.Timedelta('7 day')
result = pd.merge_asof(left = Sent, right = aggOrders, left_on = 'DataEnvio', right_on = 'ORDER_DATE', by = 'User_id', direction = 'forward', tolerance = tol)

del aggOrders, tol
result

,Campaign_id,DataEnvio,User_id,ORDER_DATE,size,sum
0,56,2021-07-01 01:15:47.000000001,16119,2021-07-02 00:00:00.000000001,2.0,3080.88
1,56,2021-07-01 01:15:47.000000001,16771,2021-07-03 00:00:00.000000001,1.0,299.90
2,56,2021-07-01 01:15:47.000000001,42704,2021-07-06 00:00:00.000000001,1.0,199.90
3,56,2021-07-01 01:15:47.000000001,18983,2021-07-06 00:00:00.000000001,1.0,122.43
4,56,2021-07-01 01:15:47.000000001,25202,2021-07-03 00:00:00.000000001,1.0,81.70
...,...,...,...,...,...,...
38395,3,2021-07-23 02:56:22.000000001,42340,NaT,NaN,NaN
38396,3,2021-07-23 02:56:22.000000001,19536,2021-07-24 00:00:00.000000001,1.0,15.92
38397,3,2021-07-23 02:56:22.000000001,40750,NaT,NaN,NaN
38398,3,2021-07-23 02:56:22.000000001,49191,NaT,NaN,NaN


In [7]:
#Necessário no pré-groupby
aggEnvios = result.groupby('Campaign_id')['Campaign_id'].size().reset_index(name = 'n_envios')

#Filtrando para atender apenas à especificação de uma única campanha por pedido. ~90s
result['time_dif'] = (result['ORDER_DATE'] - result['DataEnvio'])
result['nearest'] = False
result_filtered = pd.DataFrame([], columns = result.columns)
for gby, df in result.groupby(['User_id', 'ORDER_DATE']):   #Otimizar se der tempo.
    min_index = np.argmin(df['time_dif'])
    df.iloc[min_index, 7] = True
    result_filtered = pd.concat([result_filtered, df.copy()], axis = 0)

result_filtered.loc[result_filtered['nearest'] == False, ['size', 'sum']] = 0

del result, gby, df, min_index
result_filtered

,Campaign_id,DataEnvio,User_id,ORDER_DATE,size,sum,time_dif,nearest
1650,55,2021-07-02 18:07:34.000000001,15000,2021-07-05 00:00:00.000000001,0,0,2 days 05:52:26,False
3380,26,2021-07-03 22:55:08.000000001,15000,2021-07-05 00:00:00.000000001,1.0,119.8,1 days 01:04:52,True
3220,66,2021-07-03 20:16:32.000000001,15001,2021-07-07 00:00:00.000000001,0,0,3 days 03:43:28,False
9328,1,2021-07-06 10:05:37.000000001,15001,2021-07-07 00:00:00.000000001,1.0,23.92,0 days 13:54:23,True
35316,3,2021-07-20 02:56:22.000000001,15002,2021-07-22 00:00:00.000000001,1.0,34.7,1 days 21:03:38,True
...,...,...,...,...,...,...,...,...
24898,52,2021-07-14 01:25:08.000000001,49997,2021-07-15 00:00:00.000000001,1.0,31.96,0 days 22:34:52,True
28181,40,2021-07-16 00:26:57.000000001,49997,2021-07-17 00:00:00.000000001,0,0,0 days 23:33:03,False
28671,20,2021-07-16 06:48:14.000000001,49997,2021-07-17 00:00:00.000000001,0,0,0 days 17:11:46,False
29331,13,2021-07-16 14:04:48.000000001,49997,2021-07-17 00:00:00.000000001,1.0,326.04,0 days 09:55:12,True


In [8]:
#Para o formato final
Campanhas = Campanhas.merge(
    aggEnvios, on = 'Campaign_id', how = 'left') #n_Envios

Campanhas = Campanhas.merge(
    result_filtered.loc[result_filtered['size'] > 0, :].groupby('Campaign_id')['User_id'].agg([['n_compradores', lambda x: len(x.unique())]]).reset_index(), 
    on = 'Campaign_id', how = 'left') #n_Compradores

Campanhas = Campanhas.merge(
    result_filtered.groupby('Campaign_id')['size'].sum().reset_index(name = 'n_pedidos'), on = 'Campaign_id', how = 'left') #n_Pedidos

Campanhas = Campanhas.merge(
    result_filtered.groupby('Campaign_id')['sum'].sum().reset_index(name = '$_receita'), on = 'Campaign_id', how = 'left') #$_receita

Campanhas.head(nh)

,Campaign_id,Campaign_name,n_envios,n_compradores,n_pedidos,$_receita
0,0,Cupom_reforco_Industria,480,293,355.0,148862.42
1,1,Cupom_reforco_Distribuidores,480,286,337.0,161004.6
2,2,Incentivo1Compra_reforco_Distribuidores,480,329,391.0,184987.94


In [9]:
#Demais medidas
Campanhas['ticket_medio'] = Campanhas['$_receita'] / Campanhas['n_pedidos']
Campanhas['gasto_medio'] = Campanhas['$_receita'] / Campanhas['n_compradores']
Campanhas['tx_conversao'] = Campanhas['n_pedidos'] / Campanhas['n_compradores'] # "Pessoas impactadas" se refere aos compradores, ou todos que receberam os envios? Na dúvid fiquei com os compradores.

Campanhas.head(nh)

,Campaign_id,Campaign_name,n_envios,n_compradores,n_pedidos,$_receita,ticket_medio,gasto_medio,tx_conversao
0,0,Cupom_reforco_Industria,480,293,355.0,148862.42,419.330761,508.062867,1.211604
1,1,Cupom_reforco_Distribuidores,480,286,337.0,161004.6,477.758457,562.953147,1.178322
2,2,Incentivo1Compra_reforco_Distribuidores,480,329,391.0,184987.94,473.114936,562.273374,1.18845


In [10]:
#Para o Dash:
result_filtered['size'] = pd.to_numeric(result_filtered['size'])
result_filtered['sum'] = pd.to_numeric(result_filtered['sum'])

result_filtered.to_csv('toPBI.csv', index = False, sep = ';', decimal = ',')
pd.DataFrame({'Date' : pd.concat([result_filtered['DataEnvio'], result_filtered['ORDER_DATE']], axis = 0)}).to_csv('toPBI_date.csv', index = False, sep = ';', decimal = ',')